Notebook started March 6

Main goal: figure out how to add contiguity as a constraint/objective to PuLP



Smaller projects in service of main goal:
* dissolve county shapefile by district assignment, resulting in a new shapefile (newCDassignments) with 4 (multi)polygon entries, 1 for each district (done)
    * count how many "distinct" polygons are in each district? (no idea how?)
    * check on population/area sum values aggregated from individual counties, why they don't match pre-calculated values?
* create adjacency matrices for each district (done)
    * count connected components within each district using these (definitely possible - weird code online in various places, but maybe just brute-force with some loops, since nothing is too big?)

in this notebook, adjustments made to PuLP constraints to make things run faster:
* (only) 2 neighbors required
* 10% wiggle room on "ideal district size"

future?

* figure out plotting issues - why is ____.plot() weird in main setup, etc

* main iowa_redistricting notebook: streamline a bit?



In [41]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
from PIL import Image, ImageOps

from pulp import (LpProblem, LpMinimize, LpVariable, lpSum, 
                  PULP_CBC_CMD, GLPK_CMD, LpStatus, value) 

##           ##this stuff is necessary for our main graphs, butscrews up the bare-bones assignment graph
# from plotnine import (ggplot, aes, geom_map, geom_text, geom_label, 
#                       ggtitle, element_blank, element_rect, 
#                       scale_fill_manual, theme_minimal, theme)        


### for the plot of the merged polygons, but breaks other stuff
# import matplotlib.pyplot as plt
# plt.style.use('ggplot')
# plt.rcParams.update({'font.size': 16, 'axes.labelweight': 'bold', 'figure.figsize': (6, 6), 'axes.edgecolor': '0.2'})


## for laplacian
from scipy.sparse import csgraph

## for nullspace
from scipy.linalg import null_space

# Old stuff from main iowa_redistricting

## Prepping the census and geopandas dataframes

In [42]:
df=pd.read_csv('census.csv')
df.head()

,county_id,county,population,COUNTYFP10,latitude,longitude
0,0,Adair,7496,1,41.328528,-94.478164
1,1,Adams,3704,3,41.021656,-94.696906
2,2,Allamakee,14061,5,43.274964,-91.382751
3,3,Appanoose,12317,7,40.744683,-92.870345
4,4,Audubon,5674,9,41.679178,-94.904312


In [43]:
df['COUNTYFP10']=df['COUNTYFP10'].astype(str).str.pad(3,fillchar='0')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   county_id   99 non-null     int64  
 1   county      99 non-null     object 
 2   population  99 non-null     int64  
 3   COUNTYFP10  99 non-null     object 
 4   latitude    99 non-null     float64
 5   longitude   99 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 4.8+ KB


In [45]:
#imports county shapefiles from MGGG
shapefile_iowa = gpd.read_file('IA_counties/IA_counties.shp')
shapefile_iowa.head()

,STATEFP10,COUNTYFP10,GEOID10,NAME10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,TOTPOP,...,TOTVOT12,PRES12D,PRES12R,PRES12OTH,TOTVOT16,PRES16D,PRES16R,PRES16OTH,CD,geometry
0,19,127,19127,Marshall,Marshall County,1482770678,1803086,+42.0416910,-092.9814523,40648,...,19064,10257,8472,335,17980,7652,9146,1182,1,"POLYGON ((-92.76679 42.12346, -92.76679 42.122..."
1,19,011,19011,Benton,Benton County,1855117342,5760770,+42.0925474,-092.0576300,26076,...,14023,6862,6940,221,13844,4678,8232,934,1,"POLYGON ((-91.94773 41.86186, -91.95514 41.861..."
2,19,041,19041,Clay,Clay County,1469139214,13866941,+43.0798220,-095.1497261,16667,...,8502,3385,4951,166,8617,2249,5877,491,4,"POLYGON ((-95.26926 43.25537, -95.26140 43.255..."
3,19,165,19165,Shelby,Shelby County,1530110414,1486135,+41.6790143,-095.3089173,12167,...,6483,2469,3911,103,6370,1662,4362,346,4,"POLYGON ((-95.20902 41.86371, -95.20890 41.863..."
4,19,043,19043,Clayton,Clayton County,2016405612,36586071,+42.8409979,-091.3235108,18129,...,9138,4806,4164,168,9129,3237,5317,575,1,"POLYGON ((-91.25080 42.64558, -91.25160 42.645..."


## Merging pd and gpd dataframes and creating population heat map

### Prepping the data

In [46]:
map_population_by_county_data = shapefile_iowa.merge(df, on='COUNTYFP10')
county_populations = np.array(df['population'])
state_population = sum(county_populations)
df.sort_values('population', ascending=False).head()

,county_id,county,population,COUNTYFP10,latitude,longitude
76,76,Polk,492401,153,41.684281,-93.569720
56,56,Linn,230299,113,42.077951,-91.597673
81,81,Scott,174669,163,41.641679,-90.622290
51,51,Johnson,152854,103,41.668737,-91.588812
6,6,Black Hawk,131144,013,42.472888,-92.306059


## Creating the adjacency matrix for Iowa counties

In [47]:
shapefile_alpha=shapefile_iowa.sort_values('NAME10',ignore_index=True).copy()

In [48]:
shapefile_alpha.head()

,STATEFP10,COUNTYFP10,GEOID10,NAME10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,TOTPOP,...,TOTVOT12,PRES12D,PRES12R,PRES12OTH,TOTVOT16,PRES16D,PRES16R,PRES16OTH,CD,geometry
0,19,001,19001,Adair,Adair County,1474404167,2597997,+41.3285283,-094.4781643,7682,...,3996,1790,2114,92,3811,1133,2461,217,3,"POLYGON ((-94.35706 41.15745, -94.35992 41.157..."
1,19,003,19003,Adams,Adams County,1096700733,5353423,+41.0216555,-094.6969059,4029,...,2185,1028,1108,49,2106,565,1395,146,3,"POLYGON ((-94.81495 41.15839, -94.81268 41.158..."
2,19,005,19005,Allamakee,Allamakee County,1655214493,50995230,+43.2749637,-091.3827510,14330,...,6934,3553,3264,117,6923,2421,4093,409,1,"POLYGON ((-91.49104 43.50071, -91.49061 43.500..."
3,19,007,19007,Appanoose,Appanoose County,1287981483,49083877,+40.7446826,-092.8703450,12887,...,6245,2951,3161,133,6136,1814,4033,289,2,"POLYGON ((-93.09762 40.81197, -93.09761 40.812..."
4,19,009,19009,Audubon,Audubon County,1147264459,1152260,+41.6791780,-094.9043119,6119,...,3457,1611,1802,44,3412,1080,2136,196,4,"POLYGON ((-95.09316 41.68835, -95.09314 41.688..."


In [49]:
# full statewide adjacency matrix (first boolean, then casting as int)
#   NOTE: this operation counts a county as adjacent to itself. subtract eye(99) to get a more traditional version
#   also should do this operation after any sorting of the original shapefile

n_counties = 99

ia_adjac_matrix = pd.DataFrame()

for j in range(n_counties):
    ia_adjac_matrix[j] = shapefile_alpha.intersects(shapefile_alpha.iloc[[j]].unary_union)
ia_adjac_matrix = ia_adjac_matrix - np.identity(99)
ia_adjac_matrix = ia_adjac_matrix.astype(int)

ia_adjac_matrix_bool = ia_adjac_matrix.astype(bool)



## Optimization Model

### Creating decision variables, model, and objective function

In [50]:
#zeropadding county numbers for string PuLP variables
zp_county=np.array((df['county_id']).astype(str).str.pad(2,fillchar='0'))

In [51]:
n_districts = 4

ideal_district_size=state_population/n_districts

# poperrorbound: usually we've been doing 1%. I did 5%, 10%
poperrorbound = .1

district_maximum=int(ideal_district_size*(1+poperrorbound))
district_minimum=int(ideal_district_size*(1-poperrorbound))

In [52]:
# Create the linear programming model.
model = LpProblem("Supply-Demand-Problem", LpMinimize) 
variable_names = [zp_county[i]+str(j) for j in range(n_districts)
                                for i in range(n_counties)]
variable_names.sort() 

# The Decision Variable is 1 if the county is assigned to the district.
DV_variable_y = LpVariable.matrix("Y", variable_names, cat="Binary")
assignment = np.array(DV_variable_y).reshape(n_counties,n_districts)

# # The Decision Variable is the population allocated to the district.
DV_variable_x = LpVariable.matrix("X", variable_names, cat="Integer",
                                  lowBound=0)
allocation = np.array(DV_variable_x).reshape(n_counties,n_districts)

In [53]:
# This objective minimizes the counties split among multiple districts.
objective_function = lpSum(assignment) 
model += objective_function

### Initial Assignment / Allocation Constraints

In [54]:
# Allocate 100% of the population from each county.
for i in range(n_counties):
    model += lpSum(allocation[i][j] for j in range(n_districts)) == county_populations[i] , "Allocate All " + str(i)

In [55]:
print(allocation[0][1], county_populations[0])

X_001 7496


In [56]:
# This constraint makes assignment required for allocation.
# sum(county_populations) is the "big M"
for i in range(n_counties): 
    for j in range(n_districts):
        model += allocation[i][j] <= county_populations[i]*assignment[i][j] , "Allocation assignment of county" + str(i) + "to district" + str(j)
        model += allocation[i][j] >= .2*county_populations[i]*assignment[i][j], "Minimum allocation of county" + str(i) + "to" + str(j)

In [57]:
#numpy documentation for np array
#https://numpy.org/doc/stable/glossary.html#term-row-major
# Contiguous districts constraints

#n_neighbors: number of neighbors of a particular county required to be in the same district
#oregon was 1, we've tried 2/3
num_neighbors = 2
for j in range(n_districts):
    for i in range(n_counties):
        model += num_neighbors*assignment[i][j] <= lpSum(assignment[:,j][ia_adjac_matrix_bool[i]]), "Contiguity constraint for county " + str(i) +" in district " + str(j)



In [58]:
# District size constraints, in order to keep the size of districts by population similar
for j in range(n_districts):
    model += lpSum(allocation[i][j] for i in range(n_counties)) <= district_maximum , "District Size Maximum " + str(j)
    model += lpSum(allocation[i][j] for i in range(n_counties)) >= district_minimum , "District Size Minimum " + str(j)

In [59]:
# Only allow counties that meet certain critera to be split among multiple districts
# This not necessary for Iowa.
# for i in range(n_counties): 
#     if county_populations[i] <= 75000: 
#         model += lpSum(assignment[i][j] for j in range(n_districts)) <= 1  , "Unique Assignment for county " + str(i) 
#     else:
#         model += lpSum(assignment[i][j] for j in range(n_districts)) <= 2  , "Up-to-two Assignments for county " + str(i)

In [60]:
model.solve(GLPK_CMD(options=["--mipgap", "0.05", "--gomory"])) 

print('The model status is: ',LpStatus[model.status])
print('The objective value is: ', value(objective_function))

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/11fbe7dc451d43258ea4f66d0fc21b9b-pulp.lp -o /tmp/11fbe7dc451d43258ea4f66d0fc21b9b-pulp.sol
 --mipgap 0.05 --gomory
Reading problem data from '/tmp/11fbe7dc451d43258ea4f66d0fc21b9b-pulp.lp'...
1295 rows, 792 columns, 5520 non-zeros
792 integer variables, 396 of which are binary
3018 lines were read
GLPK Integer Optimizer 5.0
1295 rows, 792 columns, 5520 non-zeros
792 integer variables, 396 of which are binary
Preprocessing...
1295 rows, 792 columns, 5520 non-zeros
792 integer variables, 396 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  4.924e+05  ratio =  4.924e+05
GM: min|aij| =  4.346e-01  max|aij| =  2.301e+00  ratio =  5.296e+00
EQ: min|aij| =  1.905e-01  max|aij| =  1.000e+00  ratio =  5.248e+00
2N: min|aij| =  1.250e-01  max|aij| =  1.586e+00  ratio =  1.269e+01
Constructing initial basis...
Size of triangular part is 1295
Solving LP relaxation...
GLPK Simplex Optimizer

In [61]:
# Access the results
for i in range(n_counties):
    for j in range(n_districts):
        if allocation[i][j].value() > 0:
            print('County %d assigned to district %d: ' % (i, j), allocation[i][j].value())

County 0 assigned to district 3:  7496
County 1 assigned to district 1:  3704
County 2 assigned to district 1:  14061
County 3 assigned to district 0:  12317
County 4 assigned to district 3:  5674
County 5 assigned to district 3:  25575
County 6 assigned to district 2:  131144
County 7 assigned to district 1:  26715
County 8 assigned to district 2:  24988
County 9 assigned to district 2:  20565
County 10 assigned to district 3:  20823
County 11 assigned to district 0:  14334
County 12 assigned to district 0:  9927
County 13 assigned to district 3:  20760
County 14 assigned to district 1:  13127
County 15 assigned to district 1:  18505
County 16 assigned to district 2:  43127
County 17 assigned to district 3:  11658
County 18 assigned to district 3:  12012
County 19 assigned to district 1:  9748
County 20 assigned to district 3:  16384
County 21 assigned to district 1:  17043
County 22 assigned to district 3:  46460
County 23 assigned to district 1:  16525
County 24 assigned to district

## Visualizing the districts

### Prepping the data for visualization

In [62]:
# Prepare data for visualizing the results
result_value = []
for i in range(n_counties):
    for j in range(n_districts):
        var_output = {
            'County': i,
            'District': j+1,
            'Assignment': int(assignment[i][j].value()*(j+1)),
            'Allocation': allocation[i][j].value()}
        result_value.append(var_output)
        
results = pd.DataFrame(result_value)
results = results[results['Assignment'] != 0]
results = results.sort_values(['County', 'District'])


In [63]:
results

,County,District,Assignment,Allocation
3,0,4,4,7496
5,1,2,2,3704
9,2,2,2,14061
12,3,1,1,12317
19,4,4,4,5674
...,...,...,...,...
379,94,4,4,10679
383,95,4,4,20070
385,96,2,2,105941
391,97,4,4,7443


In [64]:
results = results.merge(df, left_on='County', right_on='county_id',suffixes=('_ID', '_Name'))


In [65]:
results

,County,District,Assignment,Allocation,county_id,county,population,COUNTYFP10,latitude,longitude
0,0,4,4,7496,0,Adair,7496,001,41.328528,-94.478164
1,1,2,2,3704,1,Adams,3704,003,41.021656,-94.696906
2,2,2,2,14061,2,Allamakee,14061,005,43.274964,-91.382751
3,3,1,1,12317,3,Appanoose,12317,007,40.744683,-92.870345
4,4,4,4,5674,4,Audubon,5674,009,41.679178,-94.904312
...,...,...,...,...,...,...,...,...,...,...
94,94,4,4,10679,94,Winnebago,10679,189,43.378124,-93.743488
95,95,4,4,20070,95,Winneshiek,20070,191,43.292989,-91.850788
96,96,2,2,105941,96,Woodbury,105941,193,42.393220,-96.053296
97,97,4,4,7443,97,Worth,7443,195,43.373491,-93.248533


In [66]:
data=results.drop(columns=['County', 'population']).copy()

In [67]:
data

,District,Assignment,Allocation,county_id,county,COUNTYFP10,latitude,longitude
0,4,4,7496,0,Adair,001,41.328528,-94.478164
1,2,2,3704,1,Adams,003,41.021656,-94.696906
2,2,2,14061,2,Allamakee,005,43.274964,-91.382751
3,1,1,12317,3,Appanoose,007,40.744683,-92.870345
4,4,4,5674,4,Audubon,009,41.679178,-94.904312
...,...,...,...,...,...,...,...,...
94,4,4,10679,94,Winnebago,189,43.378124,-93.743488
95,4,4,20070,95,Winneshiek,191,43.292989,-91.850788
96,2,2,105941,96,Woodbury,193,42.393220,-96.053296
97,4,4,7443,97,Worth,195,43.373491,-93.248533


# NEW STUFF HERE

## Making new adjacency matrices for districts

This can work straight with the output of the PuLP model.
Just needs "data", which is "results" minus 2 cols
(maybe a better name for the dataframe than "data"?)

In [68]:
# adjacency matrix for counties assigned to district 1
district1adj = ia_adjac_matrix.loc[data['Assignment']==1,data['Assignment']==1]
district2adj = ia_adjac_matrix.loc[data['Assignment']==2,data['Assignment']==2]
district3adj = ia_adjac_matrix.loc[data['Assignment']==3,data['Assignment']==3]
district4adj = ia_adjac_matrix.loc[data['Assignment']==4,data['Assignment']==4]


## Making Laplacian for adj mat using scipy.sparse

In [69]:
#a vector of the indices of the adjacency matrix,

dist1index = district1adj.index
dist2index = district2adj.index
dist3index = district3adj.index
dist4index = district4adj.index


######  requires new loaded thing (now at top):
# from scipy.sparse import csgraph
LaplDist1array = csgraph.laplacian(district1adj.to_numpy())
LaplDist2array = csgraph.laplacian(district2adj.to_numpy())
LaplDist3array = csgraph.laplacian(district3adj.to_numpy())
LaplDist4array = csgraph.laplacian(district4adj.to_numpy())

# convert back to a dataframe instead of an array
LaplDist1DF = pd.DataFrame(LaplDist1array,columns=dist1index,index=dist1index)
LaplDist2DF = pd.DataFrame(LaplDist2array,columns=dist2index,index=dist2index)
LaplDist3DF = pd.DataFrame(LaplDist3array,columns=dist3index,index=dist3index)
LaplDist4DF = pd.DataFrame(LaplDist4array,columns=dist4index,index=dist4index)

## calculating null space of Laplacians/counting connected components

In [71]:
LaplDist3array

array([[ 2, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  3, -1,  0,  0, -1,  0,  0,  0,  0,  0,  0],
       [-1, -1,  3,  0,  0,  0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  2,  0, -1, -1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  3,  0,  0, -1,  0, -1,  0, -1],
       [ 0, -1,  0, -1,  0,  3, -1,  0,  0,  0,  0,  0],
       [ 0,  0,  0, -1,  0, -1,  2,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  3,  0, -1,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  2,  0, -1, -1],
       [ 0,  0,  0,  0, -1,  0,  0, -1,  0,  2,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0, -1,  0,  2,  0],
       [ 0,  0,  0,  0, -1,  0,  0, -1, -1,  0,  0,  3]])

In [72]:
# from scipy.linalg import null_space

In [73]:
null_space(LaplDist3array)

array([[-0.28867513],
       [-0.28867513],
       [-0.28867513],
       [-0.28867513],
       [-0.28867513],
       [-0.28867513],
       [-0.28867513],
       [-0.28867513],
       [-0.28867513],
       [-0.28867513],
       [-0.28867513],
       [-0.28867513]])

In [78]:
# district 3 is 1 connected component, with 12 counties
null_space(LaplDist3array).shape

(12, 1)

In [94]:
# district 1 has 16 counties, in 5 distinct connected components

#look at actual null space, super ugly
# null_space(LaplDist1array)    


null_space(LaplDist1array).shape

5

In [96]:
dist1_ConnCom = null_space(LaplDist1array).shape[1]
dist2_ConnCom = null_space(LaplDist2array).shape[1]
dist3_ConnCom = null_space(LaplDist3array).shape[1]
dist4_ConnCom = null_space(LaplDist4array).shape[1]

In [101]:
ConnCom = [dist1_ConnCom, dist2_ConnCom, dist3_ConnCom, dist4_ConnCom]

In [114]:
for i in range(n_districts):
    print('District %d has %d connected component(s).' % (i+1, ConnCom[i]) )

District 1 has 5 connected component(s).
District 2 has 5 connected component(s).
District 3 has 1 connected component(s).
District 4 has 3 connected component(s).
